# Process CCP simulation results - UTC Microgrid

In [1]:
run_from_collab = False

In [2]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform
import pyarrow as pa
%matplotlib inline

In [3]:
# Import custom python file from github repo: https://changhsinlee.com/colab-import-python/
if run_from_collab:
    !pip install requests
    import requests
    # Save python as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/levorato/ccp_rtcs/master/notebooks/rccp_utils.py'
    r = requests.get(url)
    # make sure your filename is the same as how you want to import 
    with open('rccp_utils.py', 'w') as f:
        f.write(r.text)
    # now we can import
    from rccp_utils import *
else:
    from rccp_utils import *

## 1. Process result files

### 1.1. Setup project folders

In [4]:
if run_from_collab:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    gdrive_folder = '/content/gdrive/MyDrive'
else:
    gdrive_folder = '..'
print('gdrive_folder=', gdrive_folder)

gdrive_folder= ..


In [5]:
project_folder = '..'
utc_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(gdrive_folder, "rccp_experiments")
results_folder = os.path.join(gdrive_folder, "rccp_results")
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)

*** Project folder is ..
*** Instances folder is ../instances
*** Output folder is ../rccp_experiments


### 1.2. List which experiments to process

In [6]:
experiment_list = ["run_sim"]

In [7]:
experiment_folder_list = [os.path.join(output_folder, exp) for exp in experiment_list]
experiment_folder_list

['../rccp_experiments/run_sim']

### 1.3. List which CPP models to process

In [8]:
simulated_model_list = ["robust-budget", "robust-box", "robust-budget"]

### 1.4. Select instance_group to process

In [9]:
instance_group_list = ["utc-skew"]

### 1.5. Select RTCS forecast types to process

In [10]:
forecast_type_list = ["average"]  # average-based RTCS forecast

In [11]:
instances_to_process = ["A_instance2_1000s_skewed-left.txt", "A_instance2_1000s_skewed-right.txt", "A_instance2_1000s_uniform.txt",
                       "A_instance2_1NDU_Cons_1000s_skewed-left.txt", "A_instance2_1NDU_Cons_1000s_skewed-right.txt", "A_instance2_1NDU_Cons_1000s_uniform.txt",
                       "A_instance2_1NDU_Prod_1000s_skewed-left.txt", "A_instance2_1NDU_Prod_1000s_skewed-right.txt", "A_instance2_1NDU_Prod_1000s_uniform.txt",]
instance_group = "utc-skew"
instances_to_process

['A_instance2_1000s_skewed-left.txt',
 'A_instance2_1000s_skewed-right.txt',
 'A_instance2_1000s_uniform.txt',
 'A_instance2_1NDU_Cons_1000s_skewed-left.txt',
 'A_instance2_1NDU_Cons_1000s_skewed-right.txt',
 'A_instance2_1NDU_Cons_1000s_uniform.txt',
 'A_instance2_1NDU_Prod_1000s_skewed-left.txt',
 'A_instance2_1NDU_Prod_1000s_skewed-right.txt',
 'A_instance2_1NDU_Prod_1000s_uniform.txt']

In [12]:
df_list = []
for instance_name in instances_to_process:
    result_path = os.path.join(experiment_folder_list[0], "output", "simulation", "zip", instance_name)
    df_ = read_concatenated_trace_df(result_path)
    df_['InstanceName'] = instance_name
    df_list.append(df_)
df = pd.concat(df_list)

Looking for existing simulations in folder ../rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_skewed-left.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder ../rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_skewed-right.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder ../rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_uniform.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder ../rccp_experiments/run_sim/output/simulation/zip/A_instance2_1NDU_Cons_1000s_skewed-left.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder ../rccp_experiments/run_sim/output/simulation/zip/A_instance2_1NDU_Cons_1000s_skewed-right.txt ...
Concatenating individual trace_df dataframes...
Concate

In [13]:
df.head()

,Model,GammaPerc,Gamma,Strategy,Reoptimize,ModelPolicy,ForecastType,ScenarioId,t,d,ObjValue,OptTimeSpent,e_td,gap,cost,RealProcTime,InstanceName
0,robust-budget,0.0,0.0,conservative,False,ignore_model,average,477.0,1.0,1.0,7043.470732,61.100381,0.0,0.0,20.866271,3.726652,A_instance2_1000s_skewed-left.txt
1,robust-budget,0.0,0.0,conservative,False,ignore_model,average,477.0,1.0,2.0,7043.470732,0.000000,0.0,0.0,20.157530,3.727324,A_instance2_1000s_skewed-left.txt
2,robust-budget,0.0,0.0,conservative,False,ignore_model,average,477.0,1.0,3.0,7043.470732,0.000000,0.0,0.0,19.948715,3.727636,A_instance2_1000s_skewed-left.txt
3,robust-budget,0.0,0.0,conservative,False,ignore_model,average,477.0,1.0,4.0,7043.470732,0.000000,0.0,0.0,20.041516,3.728305,A_instance2_1000s_skewed-left.txt
4,robust-budget,0.0,0.0,conservative,False,ignore_model,average,477.0,1.0,5.0,7043.470732,0.000000,0.0,0.0,19.881354,3.728554,A_instance2_1000s_skewed-left.txt


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6669792 entries, 0 to 143
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Model         object 
 1   GammaPerc     float64
 2   Gamma         float64
 3   Strategy      object 
 4   Reoptimize    object 
 5   ModelPolicy   object 
 6   ForecastType  object 
 7   ScenarioId    float64
 8   t             float64
 9   d             float64
 10  ObjValue      float64
 11  OptTimeSpent  float64
 12  e_td          float64
 13  gap           float64
 14  cost          float64
 15  RealProcTime  float64
 16  InstanceName  object 
dtypes: float64(11), object(6)
memory usage: 916.0+ MB


### Create the output folders for processed results

In [15]:
reportfolder = os.path.join(results_folder, 'consolidated_results', 'df')
if not os.path.exists(reportfolder):
    os.makedirs(reportfolder)
print('Saving files on folder: ' + reportfolder)

Saving files on folder: ../rccp_results/consolidated_results/df


### Export consolidated results dataframe to pickle file


In [16]:
pickle_filepath = os.path.join(reportfolder, 'run_sim_utc_forecast_avg.results.pkl.gz')
df.to_pickle(pickle_filepath)
print('Saved consolidated results to: ', pickle_filepath)

Saved consolidated results to:  ../rccp_results/consolidated_results/df/run_sim_utc_forecast_avg.results.pkl.gz
